## 全体の流れ 
①年ごとにわけたフォルダを作成。  
②それぞれの年の中で、ダークのフォルダを作成  
③ダークのフォルダから、必要情報をまとめたデータフレームあるいは、csvファイルを作成しておく  

④天体画像を選択(いったんパス付きで指定したとする)  
⑤天体画像の日付等から、csvファイルを開き、ダークを探す  
⑥該当のダークファイルが見つかれば、そのファイル名をリストに格納。  
⑦ダーク処理を行う  
⑧COMQ画像を取得する。  

## 関数の定義

In [ ]:
import astropy.io.fits as fits
import numpy as np
import glob
import os
from scipy.ndimage import gaussian_filter
import pandas as pd
import shutil


def readheader(filename):
    return fits.open(filename)[0].header


def readdata(filename):
    return fits.open(filename)[0].data
#関数の定義
def readheader(filename):
    return fits.open(filename)[0].header

def readdata(filename):
    return fits.open(filename)[0].data

def q_bsep_posi(input):
    return input[:,::2,:,:]
def q_bsep_nega(input):
    return input[:,1::2,:,:]   
#z方向平均
def mean_z(input):
    return np.mean(input, axis=1, keepdims=True)

#gaussian filter


def gaussfilter(input):
    output = gaussian_filter(input[0, 0, :, :], sigma=6, order=0,
                             output=None, mode='nearest', cval=0.0, truncate=4.0)
    return output


def q_subch(input):
    #z方向については、COMQは1となっているはずである。
    input_dim2 = input[0, 0, :, :]
    #x軸方向に20*240と分割し、z方向に積み上げる

    divstack = np.stack(np.split(input_dim2, 16, axis=1))
    #z方向にmedianをとり、20*240をtileして320*240にする
    beforetile = np.median(divstack, axis=0)  # keepdimsは不要(二次元でいい)
    tilemedian = np.tile(beforetile, 16)
    # print(tilemedian.shape)
    input_to_output = input.copy()
    input_to_output[0, 0, :, :] -= tilemedian
    # input_to_output[0, 0, :, :] = 1
    return input_to_output

#fitsファイル作成用

def makefits(dataname, filename):
    hdu = fits.PrimaryHDU(data=dataname)
    fits.HDUList([hdu]).writeto(str(filename), overwrite=True)

## 処理の中身

In [ ]:
def process(obsfilename):
    """  
    ここには天体画像から、ダークを特定する処理を記載する。上のセル参照
    
    """
    darkheader1 =
    if (darkheader1["Q_CHAM"] != 1) or (obsheader["Q_CHAM"] != 1):
        print("CoAdd Modeが1ではない。")
    elif darkheader1["Q_CHAM"] == 1 and darkheader2["Q_CHAM"] == 1:
        darkfile1 = readdata(dark1)
        darkfile2 = readdata(dark2)
        meandark = (np.mean(dark1, axis=1, keepdims=True) +
                    np.mean(dark2, axis=1, keepdims=True)) / 2
        meandark_1 = meandark / int(darkheader1["Q_CHEB"])

        #darkの差し引き
        #Q_CHEB倍する
        obsdata = readdata(obsfilename)
        meandark_obs = meandark_1 * obsheader["Q_CHEB"]

        skydata = obsdata - meandark_obs
        skydata_p = q_bsep_posi(skydata)
        skydata_n = q_bsep_nega(skydata)

        #z方向平均
        sky_pa = mean_z(skydata_p)
        sky_na = mean_z(skydata_n)

        #gaussian　平均
        sky_paG = gaussfilter(sky_pa)
        sky_naG = gaussfilter(sky_na)

        #Flatの作成
        sky_paF = sky_pa / sky_paG
        sky_naF = sky_na / sky_naG

        #Chop差し引きした、COMQ画像に対して、Flatで割る
        comq_data = readdata(obs_comq)
        obj_obs = q_subch(comq_data)

        #Flatで割る
        obj_obs_datP0 = obj_obs / sky_naF
        obj_obs_datN0 = obj_obs / sky_paF

        makefits(obj_obs_datP0, "./out/obj" +
                 str(obsfilename[-10:-5]) + "_datP0.fits")
        makefits(obj_obs_datN0, "./out/obj" +
                 str(obsfilename[-10:-5]) + "_datN0.fits")


## 1. 年ごとにわけたフォルダの作成
データの申請段階で年ごとに行なっているので、ファイル名を変えるのみ。

## 2. ダークのフォルダを作成
これは、それぞれの年ごとのフレームリストから、該当のファイルをとってきて、フォルダを移動させる。

In [ ]:
#for example 2009年度のもの
folderpath = r'D:\2009-01-13'
judgedarkfile = r'./DataFrame/COM_2009.xlsx'

frameall = pd.read_excel(judgedarkfile)
#1行目は不要なので削除
frameall = frameall.dropna(how="all", axis=0)
darkframe = frameall[(frameall["DATA_TYP"] == "DARK") |
                     (frameall["OBJECT2"] == "DARK")]
#indexをリセット
darkframe = darkframe.reset_index(drop=True)


#filepath
filepath = r"D:\2009"
filelist = glob.glob(filepath+"\*")
#ダークのフォルダを作成。
os.makedirs(r"D:\2009\DARK", exist_ok=True)
#darkの名前を取得
for dark in darkframe["#FRAME_ID"]:
    #filename作成
    name = filepath +"\\"+ str(dark) +".fits"
    #移動
    if name in filelist:
        shutil.move(name, filepath+"\\DARK\\" + str(dark) +".fits")

## 3. ダークフォルダから、必要な情報をまとめたデータフレームを作成

In [ ]:
#for example 2009年の場合
darkfolderpath = r"D:\2009\DARK"
#darkのファイル名を取得
darkfilelist = glob.glob(darkfolderpath+"\\*")
#データフレーム作成。
darkdf = pd.DataFrame(columns=["FRRAME_ID", "DATE", "PIXTIM", "RRSTRT", "YSTRT", "CoAdd"])
for darkfile in darkfilelist:
    darkheader = readheader(darkfile)

    list1 = [darkheader["FRAMEID"], darkheader["DATE-OBS"], darkheader["Q_PIXTIM"],
            darkheader["Q_RRSTRT"], darkheader["Q_YSTRT"], darkheader["Q_CHAM"]]
    add_row = pd.DataFrame([list1], columns = darkdf.columns)
    darkdf = pd.concat([darkdf, add_row], ignore_index=True)
darkdf.to_csv(darkfolderpath + "\\" + "dark.csv")



## 4. 天体画像を与えたときにダークを探す

In [ ]:
#for example 
obsfile = 
obsheader = readheader(obsfile)